In [ ]:
import datetime
!pip install numpy
!pip install pandas
!pip install tables
import pandas as pd
!pip install tqdm
from tqdm import tqdm

In [ ]:
df = pd.read_pickle('processed_data.pkl')
training_df = df[df["train"]]
testing_df = df[df["test"]]

In [ ]:
df

In [1]:
file_names = ['prompt-variations/v1.txt', 'prompt-variations/v2.txt']
SYSTEM_PROMPT = []
for file_name in file_names:
    with open(file_name, 'r') as file:
        SYSTEM_PROMPT.append(file.read())

In [4]:
import requests
import os
from dotenv import load_dotenv
import json

load_dotenv()  # take environment variables from .env.
PERPLEXITY_API_KEY = os.getenv("PPLX_KEY")

results = []
for model in ["pplx-7b-chat", "mistral-7b-instruct",  "pplx-70b-chat", "codellama-34b-instruct", "llama-2-70b-chat",  "mixtral-8x7b-instruct"]:
    for index, row in tqdm(testing_df.iterrows(), total=testing_df["text"].count()):
        payload = {
            "model": "mistral-7b-instruct",
            "messages": [
                {
                    "role": "system",
                    "content": SYSTEM_PROMT
                },
                {
                    "role": "user",
                    "content": row["text"]
                }
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization": f"Bearer {PERPLEXITY_API_KEY}"
        }
    
        response = requests.post("https://api.perplexity.ai/chat/completions", json=payload, headers=headers)
        data = json.loads(response.text)
        answer = data['choices'][0]['message']['content']
        results.append({
            "model": model,
            "time": datetime.datetime.now(),
            "prompt": SYSTEM_PROMT,
            "text": row["text"],
            "answer": answer,
            "truth": row["hate"]
        })
        print(row["text"])
        if lines := [line for line in answer.split("\n") if "hate_speech_probability" in line]:
            print(lines)
        else:
            print(answer)

pd.DataFrame(results).to_pickle("perplexity_results.pkl")

NameError: name 'tqdm' is not defined

In [ ]:
results